# 中時電子報-財經即時 新聞爬蟲

In [51]:
import requests
from bs4 import BeautifulSoup
import os

# 輸入一個新聞專欄的主頁面，會回傳最後一頁的頁碼

In [52]:
def get_final_page(News_home_url):
    ## soup -> 取得該網頁頁面的 html 標籤
    soup = BeautifulSoup(requests.get(News_home_url).text, "lxml")

    ## 以中時電子為例，要找到最後一頁的網址 要從class為 ".last"擷取，並以"page="分割，第2個元素則為最後一頁頁碼
    # Deprecated
    # final_page = int(soup.select_one(".last").select_one("a").get("href").split("page=")[1])
    final_page = int(soup.select("a.page-link")[-1].get("href").split("page=")[1])


    return final_page

In [53]:
n = get_final_page("https://www.chinatimes.com/money/total?chdtv")

print(n)

10


# 輸入一個新聞頁面的網站，會回傳該頁面上的新聞清單

In [54]:
def get_new_list(News_page_url):
    soup = BeautifulSoup(requests.get(News_page_url).text, "lxml")
    ## 取得一頁面的全部新聞清單
    news_list = soup.select_one(".vertical-list").select("li")
    return news_list

# 輸入一個路徑，會檢查該路徑是否存在，若不在則產生資料夾

In [55]:
def check_and_mkdir(News_DB_path):
    ## os.path.isdir -> 檢查路徑是否存在
    if not(os.path.isdir(News_DB_path)):
        ## 若不存在，使用os.makedirs -> 來建立路徑資料夾
        os.makedirs(News_DB_path)

# 輸入一個新聞的網址，可以爬取標題、時間及內文，並儲存為TXT檔

In [84]:
def save_news(one_news_url):
    ## 此自訂函式會取得新聞的標題及時間，並組合為要存檔的檔名
    ## 因為檔名不能存在 #,%,*,&,|,\,/,?,>,<,:," 等符號，故如果檔名有不符合符號則取代為 _
    def get_file_name():
        new_title = soup.select_one("h1.article-title").text.strip()
        new_time = soup.select_one("time").text.strip().replace("年","_").replace("月","_").replace("日 ","_").replace(":","_")
        file_name = new_time + " " + new_title
        symbol=["#","%","*",'&','|','\\','/','?','>','<',":",'"']
        for s in file_name:
            if s in symbol:
                file_name = file_name.replace(s,"_")
        return file_name + ".txt" 
    
    
    ## 取得新聞的HTML標籤
    soup = BeautifulSoup(requests.get(one_news_url).text, "lxml")
    ## 取得要存檔的檔名
    file_name = get_file_name()
    
    ##如果檔案已經在我們資料夾中，則Pass
    if file_name in os.listdir(News_DB_path):
        #print("Pass News : ", file_name)
        # TODO: exception?
        raise Cralwer_Done
            
    else:
        ## news_content 為要寫入TXT檔的內文
        news_content = ""
        news_content += soup.select_one("h1.article-title").text.strip() + "\n" + soup.select_one("time").text.strip()

        contents = soup.select("div.article-body p")

        for c in contents:
            news_content += c.text + "\n"

        # TODO: 改成json結構
        # TODO: source or tags

        ## 最後把news_content存成txt檔，編碼為utf8
        with open(os.path.join(News_DB_path, file_name), "w", encoding="utf8") as writer:
            writer.write(news_content)

In [85]:
## test
# save_news("https://www.chinatimes.com/realtimenews/20220104004913-260410?chdtv")

# 定義爬蟲完成例外以及離開爬蟲變數

In [57]:
class Cralwer_Done(Exception):
    pass

terminal_cralwer = False
repeat = 0

# 主要程式開始

In [58]:
## 檢查資料夾路徑是否存在
News_DB_path = os.path.join("./NewsDB", "chinatimes")
check_and_mkdir(News_DB_path)
## 進入主頁
News_home_url = "https://www.chinatimes.com/money/total?chdtv"
## 取得最後一頁頁碼
final_page = get_final_page(News_home_url)
## 以迴圈讀取每一頁
for page in range(1, final_page + 1 ):
    News_page_url = "https://www.chinatimes.com/money/total?page="+str(page)+"&chdtv"
    ## 取得一頁上的新聞清單
    news_list = get_new_list(News_page_url)
    print(news_list)

    # TODO:
    # unchecked

    ## 以迴圈讀取新聞清單的每一個新聞
    # for news_el in news_list:
    #     ## 取得新聞的網址
    #     one_news_url = "https://www.chinatimes.com" + news_el.select_one("a").get("href")
    #     try:
    #         save_news(one_news_url)
    #     except Cralwer_Done:
    #         if repeat < 3:
    #             repeat+=1
    #         else:
    #             terminal_cralwer = True
    #             break
    # if terminal_cralwer:
    #     print("結束爬蟲")
    #     break

print("chinatimes crawler done !! ")

[<li>
<div class="articlebox-compact">
<div class="row">
<div class="col">
<h3 class="title"><a href="/realtimenews/20220104005296-260410">陸企頻陷債務危機 國務院：實施企業信用風險分類管理</a></h3>
<div class="meta-info">
<time datetime="2022-01-04 20:48"><span class="hour">20:48</span><span class="date">2022/01/04</span></time>
<div class="category"><a href="/money/focus">要聞</a></div>
</div>
<p class="intro">大陸近年頻繁有企業深陷債務危機面臨破產邊緣，無論國企、民企都有不少案例，涉及領域也遍布房地產、半導體、零售、汽車等。這讓大陸國務院最新宣布，將實施企業信用風險分類管理，推動監管更加公平有效。大陸央視4</p>
</div>
</div>
</div>
</li>, <li>
<div class="articlebox-compact">
<div class="row">
<div class="col-4 col-xl-3 thumb-photo-wrapper">
<div class="thumb-photo video">
<div class="cropper">
<a href="/realtimenews/20220104005262-260410">
<img alt="預期Fed提前升息 美元兌日圓攀近五年高峰" class="photo" loading="lazy" src="https://images.chinatimes.com/newsphoto/2021-12-29/330/20211229004825.jpg"/>
</a>
</div>
</div>
</div>
<div class="col">
<h3 class="title"><a href="/realtimenews/20220104005262-260410">預期Fed提前升息 美元兌日圓攀近五年高